In [1]:
import pandas as pd

import dependencies

In [2]:
path_input = dependencies.rootFolder / 'data' / 'raw'
path_output = dependencies.rootFolder / 'data' / 'processed'

## Maestro de camiones

Suponemos que es maestro de los camiones para la flota del agua va es el de la
hoja "Lista de camiones" del excel "BASE DE DATOS 2022 PRUEBA.xlsx".

Estaría bien usar el formato de la tabla de camiones del excel "Informe mensual BDT Octubre.xlsx"
Las columnas que tiene son:
- Nº
- Número BsT
- Matrícula 2010
- Numero de chasis
- Institución
- Tipo de vehiculo
- Especificacion/tipo de motor
- Fecha de compra
- Capacidad del camion
- Location
- Estado


Qué tener en cuenta:
- Qué identificador vamos a utilizar
- Que no haya matriculas repetidas
- Qué datos faltan
- Dependerá de:
    - wilayas
    - tipo de motor
    - estado


In [3]:
columns_df_trucks = [
    'number', 'registration', 'tag', 'chasis', 'institution', 'type',
     'motor_specs', 'year', 'capacity', 'location', 'status'
]

df_trucks_water = pd.concat([
    pd.read_excel(
        path_input / 'BASE DE DATOS 2022 PRUEBA.xlsx',
        sheet_name='Lista de camiones',
        usecols='A:D',
        names=['number', 'location', 'type', 'status']
    ).merge(
        pd.read_excel(
            path_input / 'BASE DE DATOS 2022 PRUEBA.xlsx',
            sheet_name='Lista de camiones',
            header=None,
            names=['number', 'tag'],
            usecols='M:N',
            skiprows=20,
            dtype={'tag': str}
        ),
        how='left',
        on='number'
    ),
    pd.DataFrame(columns=columns_df_trucks)
])

df_trucks_water['number'] = df_trucks_water['number'].str.lstrip()
df_trucks_water['number'] = df_trucks_water['number'].str.rstrip()
df_trucks_water['number'] = df_trucks_water['number'].str.upper()

df_trucks_water['location'] = df_trucks_water['location'].str.lstrip()
df_trucks_water['location'] = df_trucks_water['location'].str.rstrip()
df_trucks_water['location'] = df_trucks_water['location'].str.capitalize()

df_trucks_water['type'] = df_trucks_water['type'].str.lstrip()
df_trucks_water['type'] = df_trucks_water['type'].str.rstrip()
df_trucks_water['type'] = df_trucks_water['type'].str.capitalize()

In [4]:
df_trucks_water

,number,location,type,status,tag,registration,chasis,institution,motor_specs,year,capacity
0,CC53,Aaiun,Volvo,NaN,559915,NaN,NaN,NaN,NaN,NaN,NaN
1,CC54,Aaiun,Volvo,NaN,559916,NaN,NaN,NaN,NaN,NaN,NaN
2,CC55,Aaiun,Volvo,NaN,559917,NaN,NaN,NaN,NaN,NaN,NaN
3,CC56,Aaiun,Volvo,NaN,559918,NaN,NaN,NaN,NaN,NaN,NaN
4,CC57,Aaiun,Volvo,NaN,559919,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
79,CR219,Smara,Man,NaN,771486,NaN,NaN,NaN,NaN,NaN,NaN
80,CR223,Smara,Man,NaN,771511,NaN,NaN,NaN,NaN,NaN,NaN
81,CR224,Smara,Man,NaN,771512,NaN,NaN,NaN,NaN,NaN,NaN
82,CR225,Bojador,Man,NaN,771513,NaN,NaN,NaN,NaN,NaN,NaN


### EDA Camiones

1. Miramos si hay huecos en la columna "number"

In [5]:
df_trucks_water.loc[df_trucks_water['number'].isnull()]

,number,location,type,status,tag,registration,chasis,institution,motor_specs,year,capacity


2. Miramos si hay repetidos en la columna "number" y la columna "tag"

In [6]:
df_trucks_water.loc[df_trucks_water['number'].duplicated(keep=False)].sort_values(by='number')

,number,location,type,status,tag,registration,chasis,institution,motor_specs,year,capacity
48,CC09,Instituciones,Mercedes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,CC09,Instituciones,Mercedes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,CR140,Instituciones,Man,NaN,614224,NaN,NaN,NaN,NaN,NaN,NaN
49,CR140,Instituciones,Man,NaN,614224,NaN,NaN,NaN,NaN,NaN,NaN
34,CR223,Smara,Man,NaN,771511,NaN,NaN,NaN,NaN,NaN,NaN
80,CR223,Smara,Man,NaN,771511,NaN,NaN,NaN,NaN,NaN,NaN
35,CR224,Smara,Man,NaN,771512,NaN,NaN,NaN,NaN,NaN,NaN
81,CR224,Smara,Man,NaN,771512,NaN,NaN,NaN,NaN,NaN,NaN
50,CR225,Auserd,Man,NaN,771513,NaN,NaN,NaN,NaN,NaN,NaN
82,CR225,Bojador,Man,NaN,771513,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_trucks_water.loc[
    df_trucks_water['tag'].duplicated(keep=False) &
    df_trucks_water['tag'].notna()
].sort_values(by='number')

,number,location,type,status,tag,registration,chasis,institution,motor_specs,year,capacity
38,CR140,Instituciones,Man,NaN,614224,NaN,NaN,NaN,NaN,NaN,NaN
49,CR140,Instituciones,Man,NaN,614224,NaN,NaN,NaN,NaN,NaN,NaN
34,CR223,Smara,Man,NaN,771511,NaN,NaN,NaN,NaN,NaN,NaN
80,CR223,Smara,Man,NaN,771511,NaN,NaN,NaN,NaN,NaN,NaN
35,CR224,Smara,Man,NaN,771512,NaN,NaN,NaN,NaN,NaN,NaN
81,CR224,Smara,Man,NaN,771512,NaN,NaN,NaN,NaN,NaN,NaN
50,CR225,Auserd,Man,NaN,771513,NaN,NaN,NaN,NaN,NaN,NaN
82,CR225,Bojador,Man,NaN,771513,NaN,NaN,NaN,NaN,NaN,NaN
8,CR226,Aaiun,Man,NaN,771514,NaN,NaN,NaN,NaN,NaN,NaN
83,CR226,Dajla,Man,NaN,771514,NaN,NaN,NaN,NaN,NaN,NaN


3. Sacamos los elementos únicos de las columnas "location" y "type"

In [8]:
df_trucks_water['location'].unique()

array(['Aaiun', 'Taller central', 'Auserd', 'Bojador', 'Smara',
       'Instituciones', 'Dajla', 'Inst', 'Transporte'], dtype=object)

In [9]:
df_trucks_water['type'].unique()

array(['Volvo', 'Man', 'Mitsubishi', 'No', 'Sinotruck', 'Hundai',
       'Renault', 'Mercedes', nan, 'Iveco'], dtype=object)

## Maestro de wilayas

Hay algo que parece que puede ser un maestro de wilayas en la hoja "Lista de camiones"
del excel "BASE DE DATOS 2022 PRUEBA.xlsx".

Aún así, habrá que comparar que no hay elementos diferentes a ese maestro en las
diferentes tablas.

Las comumnas que necesitamos son
- wilaya


Qué tener en cuenta:
- Qué no hay wilayas repetidas
- Si hay cosas que no parecen wilayas, hay que preguntar qué hacer

In [10]:
columns_df_wilaya = [
    'name', 'type'
]

df_taller = pd.concat([
    pd.read_excel(
        path_input / 'BASE DE DATOS 2022 PRUEBA.xlsx',
        sheet_name='Lista de camiones',
        header=None,
        names=['type', 'name'],
        usecols='G:H',
        skiprows=49,
    ),
    pd.DataFrame(columns=columns_df_wilaya)
]).dropna(subset='name')

df_taller['name'] = df_taller['name'].str.lstrip()
df_taller['name'] = df_taller['name'].str.rstrip()
df_taller['name'] = df_taller['name'].str.capitalize()

df_taller['type'] = df_taller['type'].str.lstrip()
df_taller['type'] = df_taller['type'].str.rstrip()
df_taller['type'] = df_taller['type'].str.upper()

df_wilaya = df_taller.loc[:, ['name']]

In [11]:
df_taller

,type,name
0,TR,Aaiun
1,TR,Smara
2,TR,Auserd
3,TR,Dajla
4,TR,Bojador
5,ATC,Instituciones
6,ATC,Taller central


In [12]:
df_wilaya

,name
0,Aaiun
1,Smara
2,Auserd
3,Dajla
4,Bojador
5,Instituciones
6,Taller central


Comprobamos que las wilayas del maestro de camiones correspondan con el maestro  de wilayas

In [13]:
pd.DataFrame(df_trucks_water['location'].unique()).merge(
    df_wilaya,
    how='outer',
    left_on=0,
    right_on='name'
).loc[:, [0, 'name']].rename(
    columns={
        0: 'wilaya camiones', 'name': 'wilaya maestro'
    }
)

# pd.DataFrame(df_trucks_water['location'].unique()')

,wilaya camiones,wilaya maestro
0,Aaiun,Aaiun
1,Taller central,Taller central
2,Auserd,Auserd
3,Bojador,Bojador
4,Smara,Smara
5,Instituciones,Instituciones
6,Dajla,Dajla
7,Inst,NaN
8,Transporte,NaN


In [14]:
df_tipo_ot = pd.DataFrame(
    data={'type': ['Correctivo', 'Preventivo']}
)

In [15]:
df_tipo_ot

,type
0,Correctivo
1,Preventivo


In [16]:
columns_df_mecanico = ['mecanico', 'mecanico_2', 'mecanico_3', 'mecanico_4']

df_mecanicos_otc = pd.read_excel(
    path_input / 'BASE DE DATOS 2022 PRUEBA.xlsx',
    sheet_name='BaseDatosCorrectiva 2022',
    names=columns_df_mecanico,
    usecols='F:I',
    skiprows=9,
)

df_mecanicos_otp = pd.read_excel(
    path_input / 'BASE DE DATOS 2022 PRUEBA.xlsx',
    sheet_name='preventive BBDD',
    names=columns_df_mecanico,
    usecols='F:I',
    skiprows=5
)

df_mecanicos = pd.concat([df_mecanicos_otc, df_mecanicos_otp])

df_mecanicos = pd.DataFrame(
    pd.concat(
        [
            df_mecanicos['mecanico'],
            df_mecanicos['mecanico_2'],
            df_mecanicos['mecanico_3'],
            df_mecanicos['mecanico_4']
        ]
    ),
    columns=['mecanico']
)

df_mecanicos['mecanico'] = df_mecanicos['mecanico'].str.lstrip()
df_mecanicos['mecanico'] = df_mecanicos['mecanico'].str.rstrip()
df_mecanicos['mecanico'] = df_mecanicos['mecanico'].str.capitalize()

df_mecanicos = pd.DataFrame(
    df_mecanicos['mecanico'].unique(),
    columns=['mecanico']
)
df_mecanicos = df_mecanicos.dropna().sort_values(by='mecanico')

df_mecanicos_maestro = pd.read_excel(
    path_input / 'BASE DE DATOS 2022 PRUEBA.xlsx',
    sheet_name='Lista de camiones',
    names=['mecanico', 'taller', 'puesto'],
    usecols='I:K',
    skiprows=1
).dropna(subset=["taller"])

df_mecanicos_maestro['mecanico'] = df_mecanicos_maestro['mecanico'].str.lstrip()
df_mecanicos_maestro['mecanico'] = df_mecanicos_maestro['mecanico'].str.rstrip()
df_mecanicos_maestro['mecanico'] = df_mecanicos_maestro['mecanico'].str.capitalize()

df_mecanicos_maestro['taller'] = df_mecanicos_maestro['taller'].str.lstrip()
df_mecanicos_maestro['taller'] = df_mecanicos_maestro['taller'].str.rstrip()
df_mecanicos_maestro['taller'] = df_mecanicos_maestro['taller'].str.capitalize()

df_mecanicos_maestro['puesto'] = df_mecanicos_maestro['puesto'].str.lstrip()
df_mecanicos_maestro['puesto'] = df_mecanicos_maestro['puesto'].str.rstrip()
df_mecanicos_maestro['puesto'] = df_mecanicos_maestro['puesto'].str.capitalize()

df_mecanicos = pd.merge(
    df_mecanicos,
    df_mecanicos_maestro,
    how="left",
    on="mecanico"
)

/home/sergioretuerto/Documents/Programs/Python/venv/env/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/home/sergioretuerto/Documents/Programs/Python/venv/env/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [17]:
df_mecanicos_otc= pd.read_excel(
    path_input / 'BASE DE DATOS 2022 PRUEBA.xlsx',
    sheet_name='BaseDatosCorrectiva 2022',
    usecols='D, F:I',
    skiprows=9,
    dtype={'NºOTC': str}
)

df_mecanicos_otc = pd.melt(
    df_mecanicos_otc,
    id_vars='NºOTC',
    value_vars=['Nombre1', 'Nombre2', 'Nombre3', 'Nombre4']
).dropna(subset='value').loc[:, ['NºOTC', 'value']].rename(
    columns={
        'NºOTC': 'ot',
        'value': 'mecanico'
    }
)

df_mecanicos_otc['mecanico'] = df_mecanicos_otc['mecanico'].str.lstrip()
df_mecanicos_otc['mecanico'] = df_mecanicos_otc['mecanico'].str.rstrip()
df_mecanicos_otc['mecanico'] = df_mecanicos_otc['mecanico'].str.capitalize()

/home/sergioretuerto/Documents/Programs/Python/venv/env/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [18]:
df_mecanicos

,mecanico,taller,puesto
0,Adah,Taller general,NaN
1,Ahmed andala,NaN,NaN
2,Ahmed salek andal,Aaiun,Mecánico
3,Ali,Taller general,NaN
4,Baba lebisir,Intervencion,NaN
5,Bachir,Taller general,Jefe almacen
6,Bal-lal hammu,Dajla,Mecánico
7,Brahim,NaN,NaN
8,Brahim salem,Aused,Mecánico
9,Chaibata gailani,Taller general,NaN


In [19]:
df_tipo_averia = pd.read_excel(
    path_input / 'BASE DE DATOS 2022 PRUEBA.xlsx',
    sheet_name='BaseDatosCorrectiva 2022',
    header=None,
    usecols='S:AE',
    skiprows=9,
    nrows=1
).transpose().rename(columns={0: 'averia'})

df_otc_averias = pd.read_excel(
    path_input / 'BASE DE DATOS 2022 PRUEBA.xlsx',
    sheet_name='BaseDatosCorrectiva 2022',
    # names=['ot'],
    usecols='D, S:AE',
    skiprows=9,
    dtype={'NºOTC': str}
).rename(columns={'NºOTC': 'ot'}).dropna(subset='ot')

df_otc_averias_id = pd.melt(
    df_otc_averias,
    id_vars='ot',
    value_vars=df_tipo_averia['averia'].tolist()
).dropna(subset='value').loc[:, ['ot', 'variable']].rename(columns={'variable': 'averia'})

df_tipo_averia['averia'] = df_tipo_averia['averia'].str.lstrip()
df_tipo_averia['averia'] = df_tipo_averia['averia'].str.rstrip()
df_tipo_averia['averia'] = df_tipo_averia['averia'].str.capitalize()

df_otc_averias_id['averia'] = df_otc_averias_id['averia'].str.lstrip()
df_otc_averias_id['averia'] = df_otc_averias_id['averia'].str.rstrip()
df_otc_averias_id['averia'] = df_otc_averias_id['averia'].str.capitalize()

/home/sergioretuerto/Documents/Programs/Python/venv/env/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [20]:
df_tipo_mantenimiento = pd.read_excel(
    path_input / 'BASE DE DATOS 2022 PRUEBA.xlsx',
    sheet_name='preventive BBDD',
    header=None,
    usecols='Q:AE',
    skiprows=5,
    nrows=1
).transpose().rename(columns={0: 'mantenimiento'})

df_otp_mantenimientos = pd.read_excel(
    path_input / 'BASE DE DATOS 2022 PRUEBA.xlsx',
    sheet_name='preventive BBDD',
    usecols='C, Q:AE',
    skiprows=5,
    dtype={'OTP': str}
).rename(columns={'OTP': 'ot'}).dropna(subset='ot')

df_otp_mantenimientos_id = pd.melt(
    df_otp_mantenimientos,
    id_vars='ot',
    value_vars=df_tipo_mantenimiento['mantenimiento'].tolist()
).dropna(subset='value').rename(
    columns={
        'variable': 'mantenimiento',
        'value': 'unidades'
    }
)

df_tipo_mantenimiento['mantenimiento'] = df_tipo_mantenimiento['mantenimiento'].str.lstrip()
df_tipo_mantenimiento['mantenimiento'] = df_tipo_mantenimiento['mantenimiento'].str.rstrip()
df_tipo_mantenimiento['mantenimiento'] = df_tipo_mantenimiento['mantenimiento'].str.capitalize()

df_otp_mantenimientos_id['mantenimiento'] = df_otp_mantenimientos_id['mantenimiento'].str.lstrip()
df_otp_mantenimientos_id['mantenimiento'] = df_otp_mantenimientos_id['mantenimiento'].str.rstrip()
df_otp_mantenimientos_id['mantenimiento'] = df_otp_mantenimientos_id['mantenimiento'].str.capitalize()

/home/sergioretuerto/Documents/Programs/Python/venv/env/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [21]:
df_otp_mantenimientos_id

,ot,mantenimiento,unidades
0,507,Aceite,20.0
1,508,Aceite,20.0
2,509,Aceite,20.0
3,510,Aceite,17.0
14,521,Aceite,1.0
...,...,...,...
8299,979,Grasa,1
8300,980,Grasa,1
8304,984,Grasa,1
8305,985,Grasa,1


In [22]:
df_otc = pd.read_excel(
    path_input / 'BASE DE DATOS 2022 PRUEBA.xlsx',
    sheet_name='BaseDatosCorrectiva 2022',
    names=[
        'camion', 'wilaya', 'tipo_taller', 'ot', 'persona_solicitante',
        'fecha_inicio', 'fecha_fin', 'descripcion_solicitado', 'descripcion_realizado'
    ],
    usecols='A:E, J, L, AF, AG',
    skiprows=9,
    dtype={'ot': str}
).dropna(subset='ot').dropna(subset='camion')

df_otc['tipo_ot'] = 'Correctivo'

df_otc['ot'] = df_otc['ot'].str.lstrip()
df_otc['ot'] = df_otc['ot'].str.rstrip()

df_otc['camion'] = df_otc['camion'].str.lstrip()
df_otc['camion'] = df_otc['camion'].str.rstrip()
df_otc['camion'] = df_otc['camion'].str.upper()

df_otc['wilaya'] = df_otc['wilaya'].str.lstrip()
df_otc['wilaya'] = df_otc['wilaya'].str.rstrip()
df_otc['wilaya'] = df_otc['wilaya'].str.capitalize()

df_otc['persona_solicitante'] = df_otc['persona_solicitante'].str.lstrip()
df_otc['persona_solicitante'] = df_otc['persona_solicitante'].str.rstrip()
df_otc['persona_solicitante'] = df_otc['persona_solicitante'].str.capitalize()


df_otp = pd.read_excel(
    path_input / 'BASE DE DATOS 2022 PRUEBA.xlsx',
    sheet_name='preventive BBDD',
    names=[
        'camion', 'ot', 'tipo_mantenimiento', 'tipo_taller',
        'fecha_inicio', 'fecha_fin'
    ],
    usecols='A, C:E, J, L',
    skiprows=6,
    dtype={'ot': str}
).dropna(subset='ot').dropna(subset='camion')

df_otp['tipo_ot'] = 'Preventivo'

df_otp['ot'] = df_otp['ot'].str.lstrip()
df_otp['ot'] = df_otp['ot'].str.rstrip()

df_otp['camion'] = df_otp['camion'].str.lstrip()
df_otp['camion'] = df_otp['camion'].str.rstrip()
df_otp['camion'] = df_otp['camion'].str.upper()


df_ot = pd.concat([df_otc, df_otp])

/home/sergioretuerto/Documents/Programs/Python/venv/env/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/home/sergioretuerto/Documents/Programs/Python/venv/env/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [23]:
df_ot

,camion,wilaya,tipo_taller,ot,persona_solicitante,fecha_inicio,fecha_fin,descripcion_solicitado,descripcion_realizado,tipo_ot,tipo_mantenimiento
0,CC62,Bojador,ATC,01846,Cheibata,2022-01-03 00:00:00,2022-01-04 00:00:00,problema en los cambios,se ha cambiado la palanca,Correctivo,NaN
1,CC77,Smara,TR,01847,Chofer,2022-01-04 00:00:00,2022-01-05 00:00:00,neumatico en mal estado,se ha cambiado el neumatico por uno nuevo,Correctivo,NaN
2,CC05,Smara,TR,01848,Chofer,2022-01-09 00:00:00,2022-01-10 00:00:00,neumatico en mal estado,se ha cambiado el neumatico por uno nuevo,Correctivo,NaN
3,CC08,Smara,TR,01849,Chofer,2022-01-09 00:00:00,2022-01-09 00:00:00,rutura en las rempo olas,se han soldado las rompo olas de la cisterna,Correctivo,NaN
4,CC08,Smara,TR,01850,Chofer,2022-01-06 00:00:00,2022-01-06 00:00:00,neumatico deteriorado,se ha cambiado el neumatico por uno nuevo,Correctivo,NaN
...,...,...,...,...,...,...,...,...,...,...,...
488,CT3,NaN,TR,995,NaN,2022-09-13 00:00:00,2022-09-13 00:00:00,NaN,NaN,Preventivo,Mensual
489,CT4,NaN,TR,996,NaN,2022-09-17 00:00:00,2022-09-17 00:00:00,NaN,NaN,Preventivo,Mensual
490,CT5,NaN,TR,997,NaN,2022-09-15 00:00:00,2022-09-15 00:00:00,NaN,NaN,Preventivo,Mensual
491,CT6,NaN,TR,998,NaN,2022-09-25 00:00:00,2022-09-25 00:00:00,NaN,NaN,Preventivo,Mensual


In [64]:
import numpy as np


df_disponibilidad = pd.DataFrame()

for index, row in df_ot.dropna(subset=['fecha_inicio', 'fecha_fin']).iterrows():
    if type(row['fecha_inicio']) == str or type(row['fecha_fin']) == str:
        print('Vacios')
    else:
        df_disponibilidad_ot = pd.date_range(
            row['fecha_inicio'],
            row['fecha_fin'],
            freq='d').to_frame(name='date').reset_index(drop=True)
        df_disponibilidad_ot['camion'] = row['camion']
        df_disponibilidad_ot['disponibilidad'] = 1

        df_disponibilidad = pd.concat([df_disponibilidad, df_disponibilidad_ot])

df_disponibilidad = df_disponibilidad.reset_index(drop=True)
        

Vacios
Vacios
Vacios
Vacios


In [118]:
df_wilaya.to_csv(path_output / 'wilaya.csv', index=False)
df_mecanicos.to_csv(path_output / 'mecanico.csv')
df_trucks_water.to_csv(path_output / 'camion.csv')
df_taller.to_csv(path_output / 'taller.csv')
df_tipo_ot.to_csv(path_output / 'tipo_ot.csv')
df_mecanicos_otc.to_csv(path_output / 'mecanico-ot.csv')
df_tipo_averia.to_csv(path_output / 'averia.csv')
df_otc_averias_id.to_csv(path_output / 'averia-ot.csv')
df_tipo_mantenimiento.to_csv(path_output / 'repuesto.csv')
df_otp_mantenimientos_id.to_csv(path_output / 'repuesto-ot.csv')
df_ot.to_csv(path_output / 'ot.csv')
df_disponibilidad.to_csv(path_output / 'disponibilidad.csv')